The following tutorial is available on the [Wallaroo Github Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2026.1_tutorials/wallaroo-run-anywhere/inference-on-any-architecture/x86/edge-computer-vision-yolov8).

## Computer Vision Yolov8n Edge Deployment in Wallaroo

The [Yolov8](https://github.com/ultralytics/ultralytics) computer vision model is used for fast recognition of objects in images.  This tutorial demonstrates how to deploy a Yolov8n pre-trained model into a Wallaroo Ops server and perform inferences on it.

Wallaroo Ops Center provides the ability to publish Wallaroo pipelines to an Open Continer Initative (OCI) compliant registry, then deploy those pipelines on edge devices as Docker container or Kubernetes pods.  See [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/) for full details.

For this tutorial, the helper module `CVDemoUtils` and `WallarooUtils` are used to transform a sample image into a pandas DataFrame.  This DataFrame is then submitted to the Yolov8n model deployed in Wallaroo.

This demonstration follows these steps:

* In Wallaroo Ops:
  * Upload the Yolo8 model to Wallaroo Ops
  * Add the Yolo8 model as a Wallaroo pipeline step
  * Deploy the Wallaroo pipeline and allocate cluster resources to the pipeline
  * Perform sample inferences
  * Undeploy and return the resources to the cluster.
  * Publish the pipeline to the OCI registry configured in the Wallaroo Ops server.
* In a remote aka edge device:
  * Deploy the published pipeline as a Wallaroo Inference Server on an edge device through Docker.

## Tutorial Notes

To run this tutorial in the **Wallaroo JupyterHub Service**, import the `tensorflow-cpu` library by executing the following command in the terminal shell:

```bash
pip install tensorflow-cpu==2.13.1 --user
```

Then proceed with the tutorial.  This **only** applies to running this tutorial in Wallaroo's JupyterHub service, and does **not** affect model upload and packaging in Wallaroo.

## References

* [Wallaroo Workspaces](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/):  Workspaces are environments were users upload models, create pipelines and other artifacts.  The workspace should be considered the fundamental area where work is done.  Workspaces are shared with other users to give them access to the same models, pipelines, etc.
* [Wallaroo Model Upload and Registration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/): ML Models are uploaded to Wallaroo through the SDK or the MLOps API to a **workspace**.  ML models include default runtimes (ONNX, Python Step, and TensorFlow) that are run directly through the Wallaroo engine, and containerized runtimes (Hugging Face, PyTorch, etc) that are run through in a container through the Wallaroo engine.
* [Wallaroo Pipelines](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/): Pipelines are used to deploy models for inferencing.  Each model is a **pipeline step** in a pipelines, where the inputs of the previous step are fed into the next.  Pipeline steps can be ML models, Python scripts, or Custom Model (these contain necessary models and artifacts for running a model).
* [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/): Details on publishing a Wallaroo pipeline to an OCI Registry and deploying it as a Wallaroo Server instance.

## Data Scientist Steps

The following details the steps a Data Scientist performs in uploading and verifying the model in a Wallaroo Ops server.

### Load Libraries

The first step is loading the required libraries including the [Wallaroo Python module](https://pypi.org/project/wallaroo/).

In [1]:
# Import Wallaroo Python SDK
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
from CVDemoUtils import CVDemo
from WallarooUtils import Util
cvDemo = CVDemo()
util = Util()

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)


### Connect to the Wallaroo Instance through the User Interface

The next step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client(api_endpoint="https://autoscale-uat-gcp.wallaroo.dev",
                     auth_type="sso")

### Create a New Workspace

We'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up variables for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

To allow this tutorial to be run by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.  Feel free to set `suffix=''` if this is not required.

In [3]:
model_name = 'yolov8n'
model_filename = './models/yolov8n.onnx'
pipeline_name = 'yolo8demonstration'
workspace_name = f'yolo8-edge-demonstration'


In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

{'name': 'yolo8-edge-demonstration', 'id': 1668, 'archived': False, 'created_by': 'john.hummel@wallaroo.ai', 'created_at': '2025-05-16T15:35:07.27544+00:00', 'models': [{'name': 'yolov8n', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 7, 11, 19, 8, 50, 376616, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 5, 16, 15, 35, 8, 345938, tzinfo=tzutc())}], 'pipelines': [{'name': 'yolo8demonstration', 'create_time': datetime.datetime(2025, 5, 16, 15, 35, 9, 971363, tzinfo=tzutc()), 'definition': '[]'}]}

### Upload the Model

When a model is uploaded to a Wallaroo cluster, it is optimized and packaged to make it ready to run as part of a pipeline. In many times, the Wallaroo Server can natively run a model without any Python overhead. In other cases, such as a Python script, a custom Python environment will be automatically generated. This is comparable to the process of "containerizing" a model by adding a small HTTP server and other wrapping around it.

Our pretrained model is in ONNX format, which is specified in the `framework` parameter.  For this model, the tensor fields are set to `images` to match the input parameters, and the batch configuration is set to `single` - only one record will be submitted at a time.

In [5]:
# Upload Retrained Yolo8 Model 
yolov8_model = (wl.upload_model(model_name, 
                               model_filename, 
                               framework=Framework.ONNX)
                               .configure(tensor_fields=['images'],
                                          batch_config="single"
                                          )
                )

### Pipeline Deployment Configuration

For our pipeline we set the deployment configuration to only use 1 cpu and 1 GiB of RAM.

In [6]:
deployment_config = wallaroo.DeploymentConfigBuilder() \
                    .replica_count(1) \
                    .cpus(1) \
                    .memory("1Gi") \
                    .build()

### Build and Deploy the Pipeline

Now we build our pipeline and set our Yolo8 model as a pipeline step, then deploy the pipeline using the deployment configuration above.

In [7]:
pipeline = wl.build_pipeline(pipeline_name) \
            .add_model_step(yolov8_model)        

In [8]:
pipeline.deploy(deployment_config=deployment_config)

name,yolo8demonstration
created,2025-05-16 15:35:09.971363+00:00
last_updated,2025-07-16 16:29:05.475468+00:00
deployed,True
workspace_id,1668
workspace_name,yolo8-edge-demonstration
arch,x86
accel,none
tags,
versions,"1ef082f2-944b-41a4-ae79-38d5c3ba792c, 3ef64771-7147-41d5-aafd-082d89d1c604, 94cdcfed-d628-4031-aab6-3214b67965b4, 18c0f32f-f9cf-48a8-96cc-3cd6e574780e, 870b3f92-6cbd-469a-8b0d-88a04c6541fb, bd733642-9948-4619-a90e-18dc5720e6a4, e2dcb8a0-389e-4f85-8363-8636fee4dc37, a02cc27b-2e69-41df-a320-fbc49783e7af"
steps,yolov8n


### Convert Image to DataFrame

The sample image `dogbike.png` was converted to a DataFrame using the `cvDemo` helper modules.  The converted DataFrame is stored as `./data/dogbike.df.json` to save time.

The code sample below demonstrates how to use this module to convert the sample image to a DataFrame.

```python
# convert the image to a tensor

width, height = 640, 640
tensor1, resizedImage1 = cvDemo.loadImageAndResize('dogbike.png', width, height)
tensor1.flatten()

# add the tensor to a DataFrame and save the DataFrame in pandas record format
df = util.convert_data(tensor1,'images')
df.to_json("data.json", orient = 'records')
```

### Inference Request

We submit the DataFrame to the pipeline using `wallaroo.pipeline.infer`, and store the results in the variable `inf1`.  A copy of the dataframe is stored in the file `./data/dogbike.df.json`.

In [9]:
width, height = 640, 480
tensor1, resizedImage1 = cvDemo.loadImageAndResize('./data/dogbike.png', width, height)

inf1 = pipeline.infer_from_file('./data/dogbike.df.json')

### Inference Through Pipeline API

Another method of performing an inference using the pipeline's deployment url.

Performing an inference through an API requires the following:

* The authentication token to authorize the connection to the pipeline.
* The pipeline's inference URL.
* Inference data to sent to the pipeline - in JSON, DataFrame records format, or Apache Arrow.

Full details are available through the [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/) on how retrieve an authorization token and perform inferences through the pipeline's API.

For this demonstration we'll submit the pandas record, request a pandas record as the return, and set the authorization header.  The results will be stored in the file `curl_response.df`.

In [10]:
deploy_url = pipeline._deployment._url()

headers = wl.auth.auth_header()

headers['Content-Type']='application/json; format=pandas-records'
headers['Accept']='application/json; format=pandas-records'

In [11]:
!curl -X POST {deploy_url} \
    -H "Authorization:{headers['Authorization']}" \
    -H "Content-Type:application/json; format=pandas-records" \
    -H "Accept:application/json; format=pandas-records" \
    --data @./data/dogbike.df.json > curl_response.df

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38.0M  100 22.9M  100 15.0M  4978k  3277k  0:00:04  0:00:04 --:--:-- 9086k


In [13]:
pipeline.undeploy()

name,yolo8demonstration
created,2025-05-16 15:35:09.971363+00:00
last_updated,2025-07-16 16:29:05.475468+00:00
deployed,False
workspace_id,1668
workspace_name,yolo8-edge-demonstration
arch,x86
accel,none
tags,
versions,"1ef082f2-944b-41a4-ae79-38d5c3ba792c, 3ef64771-7147-41d5-aafd-082d89d1c604, 94cdcfed-d628-4031-aab6-3214b67965b4, 18c0f32f-f9cf-48a8-96cc-3cd6e574780e, 870b3f92-6cbd-469a-8b0d-88a04c6541fb, bd733642-9948-4619-a90e-18dc5720e6a4, e2dcb8a0-389e-4f85-8363-8636fee4dc37, a02cc27b-2e69-41df-a320-fbc49783e7af"
steps,yolov8n


### Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)`.

### Publish Example

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In [14]:
pub = pipeline.publish(deployment_config)
pub

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing.... Published.


ID,108
Pipeline Name,yolo8demonstration
Pipeline Version,afff78fe-0a51-483d-8ea5-d82bce235d58
Status,Published
Workspace Id,1668
Workspace Name,yolo8-edge-demonstration
Edges,
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/yolo8demonstration:afff78fe-0a51-483d-8ea5-d82bce235d58
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/yolo8demonstration
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:e5374f5842abd4c65716b7f6b88d190b07cd13e53db42f67c412a3d08a36aed3


### List Published Pipeline

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

In [15]:
wl.list_pipelines(workspace_name=workspace_name)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
yolo8demonstration,2025-16-May 15:35:09,2025-16-Jul 16:36:45,False,1668,yolo8-edge-demonstration,x86,none,,"afff78fe-0a51-483d-8ea5-d82bce235d58, 1ef082f2-944b-41a4-ae79-38d5c3ba792c, 3ef64771-7147-41d5-aafd-082d89d1c604, 94cdcfed-d628-4031-aab6-3214b67965b4, 18c0f32f-f9cf-48a8-96cc-3cd6e574780e, 870b3f92-6cbd-469a-8b0d-88a04c6541fb, bd733642-9948-4619-a90e-18dc5720e6a4, e2dcb8a0-389e-4f85-8363-8636fee4dc37, a02cc27b-2e69-41df-a320-fbc49783e7af",yolov8n,True


### List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

#### List Publishes Parameters

N/A

#### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

In [16]:
pipeline.publishes()

id,Pipeline Name,Pipeline Version,Workspace Id,Workspace Name,Edges,Engine URL,Pipeline URL,Created By,Created At,Updated At
50,yolo8demonstration,bd733642-9948-4619-a90e-18dc5720e6a4,1668,yolo8-edge-demonstration,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6139,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/yolo8demonstration:bd733642-9948-4619-a90e-18dc5720e6a4,john.hummel@wallaroo.ai,2025-16-May 15:40:18,2025-16-May 15:40:18
83,yolo8demonstration,94cdcfed-d628-4031-aab6-3214b67965b4,1668,yolo8-edge-demonstration,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/yolo8demonstration:94cdcfed-d628-4031-aab6-3214b67965b4,john.hummel@wallaroo.ai,2025-11-Jul 19:09:12,2025-11-Jul 19:09:12
108,yolo8demonstration,afff78fe-0a51-483d-8ea5-d82bce235d58,1668,yolo8-edge-demonstration,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/yolo8demonstration:afff78fe-0a51-483d-8ea5-d82bce235d58,john.hummel@wallaroo.ai,2025-16-Jul 16:36:47,2025-16-Jul 16:36:47


## DevOps - Pipeline Edge Deployment

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer.  For our example, we will use the `docker run` command output during the pipeline publish.

## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

```bash
curl localhost:8080/pipelines
{"pipelines":[{"id":"yolo8demonstration","status":"Running"}]}
```

The following example uses the host `localhost`.  Replace with your own host name of your Edge deployed pipeline.

In [20]:
!curl localhost:8080/pipelines

{"pipelines":[{"id":"yolo8demonstration","status":"Running"}]}

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

```bash
{"models":[{"name":"yolov8n","sha":"3ed5cd199e0e6e419bd3d474cf74f2e378aacbf586e40f24d1f8c89c2c476a08","status":"Running","version":"7af40d06-d18f-4b3f-9dd3-0a15248f01c8"}]}
```

The following example uses the host `localhost`.  Replace with your own host name of your Edge deployed pipeline.

In [21]:
!curl localhost:8080/models

{"models":[{"name":"yolov8n","sha":"3ed5cd199e0e6e419bd3d474cf74f2e378aacbf586e40f24d1f8c89c2c476a08","status":"Running","version":"7af40d06-d18f-4b3f-9dd3-0a15248f01c8"}]}

### Edge Inference Endpoint

The inference endpoint takes the following pattern:

* `/pipelines/infer`

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/#anomaly-testing).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.
  
Once deployed, we can perform an inference through the deployment URL.  We'll assume we're running the inference request through the localhost and submitting the local file `./data/dogbike.df.json`.  Note that our inference endpoint is `pipelines/yolo8demonstration` - the same as our pipeline name.

The following example demonstrates sending an inference request to the edge deployed pipeline and storing the results in a pandas DataFrame in record format.  The results can then be exported to other processes to render the detected images or other use cases.

In [ ]:
!curl -X POST localhost:8080/infer \
    -H "Content-Type: application/json; format=pandas-records" \
    --data @./data/dogbike.df.json > edge-results.df.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 28.7M  100 13.6M  100 15.0M  14.2M  15.7M --:--:-- --:--:-- --:--:-- 30.3M
